In [1]:
import random
import timeit
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests as req
import json

from pprint import pprint as p
from citipy import citipy
from datetime import datetime as dt


from config import g_api_key
from config import weather_api_key

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [2]:
def lats(size_):
    lats = []
    lats = np.random.uniform(low=-90, high=90, size=size_)
    return lats

def lngs(size_):
    lngs = []
    lngs = np.random.uniform(low=-180, high=180, size=size_)
    return lngs

lats_lngs = zip(lats(2000), lngs(2000))

coords = list(lats_lngs)
coords

[(-86.64318270735416, 105.77598253814159),
 (-16.415556425706257, -32.7925375507584),
 (35.36122613560188, -66.18876222715315),
 (-49.710689522394844, -164.4361958441514),
 (29.9150891853443, 85.67904017054144),
 (81.20756450144862, 135.59513317405185),
 (-80.566120006536, 75.52943235873033),
 (-16.638378953266113, -7.2012152245760035),
 (-65.96277323777177, -134.57966663114385),
 (-3.385503482569277, 110.1701876281137),
 (22.39875523525673, -3.7192900844973735),
 (-60.21291045814155, -59.13617653421521),
 (-83.77808804887385, 5.616923135691337),
 (-30.90722013192439, -94.62894706973833),
 (42.922042758466006, 35.85183921384683),
 (-34.31641900613702, 11.02994422629314),
 (68.72321498351712, 23.1914190849281),
 (-26.453543129999133, 48.35408879246961),
 (67.95002588198216, 86.87413574431525),
 (-65.34913281368028, 158.99480079577796),
 (31.943877373509153, -164.371654506514),
 (37.49292060198438, -132.53629794323953),
 (-46.142322741181445, 39.90685986908542),
 (67.60905850513049, -81.

In [3]:
cities = []

for coordinate in coords:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)
    
len(cities)
cities

['albany',
 'belmonte',
 'saint george',
 'avarua',
 'kathmandu',
 'nizhneyansk',
 'busselton',
 'jamestown',
 'rikitea',
 'pangkalanbuun',
 'taoudenni',
 'ushuaia',
 'hermanus',
 'lebu',
 'sinop',
 'saldanha',
 'kautokeino',
 'taolanaro',
 'igarka',
 'bluff',
 'kapaa',
 'fortuna',
 'east london',
 'clyde river',
 'fort-shevchenko',
 'kamenskoye',
 'kieta',
 'ghanzi',
 'palabuhanratu',
 'azimur',
 'iqaluit',
 'illoqqortoormiut',
 'luganville',
 'punta alta',
 'vaini',
 'mar del plata',
 'mataura',
 'dwarka',
 'port hedland',
 'arlit',
 'provideniya',
 'talara',
 'longyearbyen',
 'caucaia',
 'tuktoyaktuk',
 'fukue',
 'upernavik',
 'oksbol',
 'carnarvon',
 'port-gentil',
 'kodiak',
 'sentyabrskiy',
 'gimli',
 'encruzilhada do sul',
 'yellowknife',
 'cape town',
 'malanje',
 'sorland',
 'nikolskoye',
 'nanortalik',
 'atuona',
 'bredasdorp',
 'altea',
 'muros',
 'sambava',
 'narsaq',
 'tuatapere',
 'attawapiskat',
 'umm lajj',
 'ayan',
 'tanda',
 'umba',
 'amderma',
 'kulaman',
 'kysyl-syr

In [4]:
# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [5]:
# Loop through all the cities in our list
for i, city in enumerate(cities):
    ### 60 calls per minute
    time.sleep(2)
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
#    print(city_url)
#    break
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        city_weather = req.get(city_url).json()
        #p(city_weather)        
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        try:
            city_rain_inches = city_weather["rain"]["1h"]
        except KeyError:
            city_rain_inches = 0
        try:
            city_snow_inches = city_weather["snow"]["1h"]
        except KeyError:
            city_snow_inches = 0
            # Append the City information into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Rain (inches)": city_rain_inches,
                          "Snow (inches)": city_snow_inches,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | albany
Processing Record 2 of Set 1 | belmonte
Processing Record 3 of Set 1 | saint george
Processing Record 4 of Set 1 | avarua
Processing Record 5 of Set 1 | kathmandu
Processing Record 6 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 7 of Set 1 | busselton
Processing Record 8 of Set 1 | jamestown
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | pangkalanbuun
Processing Record 11 of Set 1 | taoudenni
Processing Record 12 of Set 1 | ushuaia
Processing Record 13 of Set 1 | hermanus
Processing Record 14 of Set 1 | lebu
Processing Record 15 of Set 1 | sinop
Processing Record 16 of Set 1 | saldanha
Processing Record 17 of Set 1 | kautokeino
Processing Record 18 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 19 of Set 1 | igarka
Processing Record 20 of Set 1 | bluff
Processing Record 21 of Set 1 | kapaa
Processing Record 22 of Set 1 | fortuna
Processing Record 23 of Set 1 | east london
Processing

In [6]:
len(city_data)

696

In [7]:
# convert to data frame
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Current Description,Rain (inches),Snow (inches),Humidity,Cloudiness,Wind Speed,Country
0,Albany,42.6001,-73.9662,11.84,light snow,0.00,0.29,81,100,1.99,US
1,Belmonte,-15.8631,-38.8828,77.50,moderate rain,2.05,0.00,67,43,7.96,BR
2,Saint George,37.1041,-113.5841,50.47,clear sky,0.00,0.00,46,0,0.00,US
3,Avarua,-21.2078,-159.7750,75.25,scattered clouds,0.00,0.00,94,45,3.44,CK
4,Kathmandu,27.7167,85.3167,46.62,few clouds,0.00,0.00,76,20,4.61,NP
...,...,...,...,...,...,...,...,...,...,...,...
691,Ati,13.2154,18.3353,81.73,scattered clouds,0.00,0.00,11,36,11.05,TD
692,Mumbwa,-14.9783,27.0619,69.28,moderate rain,2.99,0.00,92,98,4.12,ZM
693,Vaitape,-16.5167,-151.7500,80.04,few clouds,0.00,0.00,75,12,10.29,PF
694,Boyolangu,-8.1181,111.8935,76.33,overcast clouds,0.00,0.00,89,100,1.07,ID


In [8]:
# Create the output File (CSV)
output_data_file = "Resources/WeatherPy_Database.csv"

# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")